In [152]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, date, datetime
import difflib
import json
import torch
import torch.nn as nn
# from ml_model import cols, family_mapper, model_mapper, dev, model
from queries import kanban_storage, proc, update_processing, get_kanban, get_batch, set_kanban, stock, stock_all, demand_all, demand
from util_func import ProcessingStorage, StockStorage
set_kanban()

set_kanban


In [2]:
class SimpleNet(nn.Module):
    def __init__(self, dimension):
        super().__init__()
        self.linear1 = nn.Linear(dimension, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64, 1)

        self.act1 = nn.ReLU()  # Activation function
        self.act2 = nn.ReLU()  # Activation function
        self.act3 = nn.ReLU()  # Activation function

    # Perform the computation
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act2(x)
        x = self.linear3(x)
        x = self.act3(x)
        x = self.linear4(x)
        return x
ALL_ML_MODEL_PATH = './ml/ALL_26_13.pth'
dimension = 59
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

model = SimpleNet(dimension).to(dev)
if torch.cuda.is_available():
    model.load_state_dict(torch.load(ALL_ML_MODEL_PATH))
else:
    model.load_state_dict(torch.load(
        ALL_ML_MODEL_PATH, map_location=torch.device('cpu')))

In [23]:
# proc = ProcessingStorage()
# proc.
# dimension = 59
# ALL_ML_MODEL_PATH = 'ALL_26_13.pth'
FAMILY = ['    SR', 'HCR', 'HDR', 'HRW',
          'HSR', 'SHS', 'SHW', 'SR', 'SRS', 'SRS-W']
MODEL = ['12', '12W', '15', '15/300R', '15W', '17', '20', '21',
         '25', '27', '30', '35', '45', '45/800R', '55', '65', '9', '9X']
REFRENCE = ['GY3', 'GY4', 'GYHAAS', 'HK']
facilities = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020']
cols_to_keep = ['B0012', 'B0015', 'B0021', 'B0025', 'B0030', 'B0040', 'B0050', 'B0060', 'B0070', 'B0080', 'C0020', 'C0030', 'C0031', 'C0035', 'C0040', 'C0050', 'C0060', 'C0070', 'C0075', 'C0080', 'C0090', 'C0100', 'C0110', 'C0120', 'D0010', 'D0020', 'F0010', 'F0020', 'GL010', 'GL030', 'GL130', 'GL140', 'H0010', 'H0020', 'H0030', 'H0045', 'H0046', 'H0050', 'H0051', 'H0060', 'H0070', 'J0020', 'J0025', 'J0050', 'J0060', 'K0010', 'K0011', 'K0020', 'GK', 'GP', 'LE', 'LT', 'FULL', 'order_quantity', 'product_family', 'product_length', 'product_model', 'reference_number', 'steps']
flag_list = ['GK', 'GP', 'LE', 'LT']

family_map = {}
for i, v in enumerate(FAMILY):
    family_map[v] = i
model_map = {}
for i, v in enumerate(MODEL):
    model_map[v] = i
reference_map = {}
for i, v in enumerate(REFRENCE):
    reference_map[v] = i
def family_mapper(x):
    try:
        return family_map[x]
    except Exception as e:
        print("family:", x, e)
def model_mapper(x):
    try:
        return model_map[x]
    except Exception as e:
        print("model", x, e)


def reference_mapper(x):
    try:
        if x.startswith('HK'): return reference_map['HK']
        else:
            return reference_map[x]
    except Exception as e:
        print("reference_number", x, e)
def all_priority_call_test():
    run_id, batch_order, batch_df = get_batch()
    response = calc_priority(batch_order)
    response.set_index("batch_id", inplace=True)
    result = pd.concat([response, batch_df], axis=1, join='outer')
    result = result.reset_index()
    return result.to_json(orient="records")


def calc_priority(batch_order):
    if proc.empty:
        update_processing()
    gy_processing = proc.BACK_PROCESS.copy()
    gy_processing.drop(columns=['order_scheduled_due'], inplace=True)
    gy_prediction = partial_predict_backprocess(gy_processing)
    hk_prediction = partial_predict_hk(proc.HK.copy())

    kanban_df = get_kanban()
    kanban_df = kanban_df.set_index('des')
    has_start_due = kanban_df[kanban_df.start_due.notna()]
    has_start_due_list = has_start_due.index.to_list()
    for row in gy_prediction.itertuples():
        des = row.item_description
        if row.item_description in has_start_due_list:
            gy_prediction.loc[row[0],
                              'start_due'] = has_start_due.loc[des].start_due - row.pred

    gy_mos = gy_prediction.index.to_list()
    hk_mos = hk_prediction.index.to_list()

    # mathcing HK mos and batch_id
    for row in batch_order.itertuples():
        index = row[0]
        if index in hk_mos:
            batch_order.loc[index,
                            "start_due"] = hk_prediction.loc[index]['start_due']
    # mathcing GY3 mos and batch_id
    empty_batch_order = batch_order[batch_order['start_due'].isna()]
    for row in empty_batch_order.itertuples():
        index = row[0]
        if index in gy_mos:
            batch_order.loc[index,
                            "start_due"] = gy_prediction.loc[index]['start_due']
    batch_order = batch_order[batch_order['start_due'].notna()]
    batch_order = batch_order.sort_values(by='start_due')
    batch_order = batch_order.drop_duplicates('batch_id')
    batch_order = batch_order.reset_index().reset_index()
    batch_order = batch_order.rename(columns={"index": "priority"})
    batch_order['priority'] = batch_order['priority'].apply(np.int64)
    return batch_order


In [ ]:
def partial_predict_backprocess(df):    
    df.set_index('order_number', inplace=True)
    res = df.copy()    
    with open('./etc/stage_mapper3.json') as json_file:
        stage_mapper = json.load(json_file)
        facility_index = np.array(stage_mapper["index"])
        facility_map = stage_mapper["map"]
        for row in res.itertuples():
            index = row[0]
            facility_id = row.facility_id
            facilities = np.array(row.facility.split(' '))
            cur_facility_index = np.where(facilities == facility_id)[0][0]
            mask = np.isin(facility_index, facilities)
            f_key = str(np.where(mask == True)[0])
            if cur_facility_index == 1:
                df.loc[index, 'ratio'] = 1
            else:
                ratio_arr = None
                if f_key in facility_map.keys():
                    ratio_arr = facility_map[f_key]
                else:
                    sub_key = difflib.get_close_matches(
                        f_key, facility_map.keys())[0]
                    ratio_arr = facility_map[sub_key]
                df.loc[index, 'ratio'] = sum(
                    ratio_arr[cur_facility_index - 1:])
            for f in facilities:
                res.loc[index, f] = 1
    for name in flag_list:
        res[name] = np.where(res['item_description'].str.count(name), 1, 0)
    res['FULL'] = False
    res['product_family'] = res['product_family'].apply(
        lambda x: family_mapper(x))
    res['product_model'] = res['product_model'].apply(
        lambda x: model_mapper(x))
    res['reference_number'] = res['reference_number'].apply(
        lambda x: reference_mapper(x))
    res['product_length'] = res['product_length'].apply(lambda x: x / 1000)
    res['steps'] = res['facility'].apply(lambda x: len(x.split(' ')))
    res = res.reindex(set(res.columns.tolist() + cols_to_keep), axis=1)  # version > 0.20.0
    res = res[cols_to_keep]
    res = res.fillna(0)
    val = res.astype(float)
    x = val.iloc[:, :].values
    v_xb = torch.tensor(x, dtype=torch.float32).to(dev)
    pred = model(v_xb)[:, 0].tolist()
    df['original_pred'] = pred / 24
    df['pred'] = pred * df['ratio']
    df['pred'] = df['pred'].astype('int')
    df['pred'] = df['pred'].apply(lambda x: timedelta(hours=x))
    df = df.drop_duplicates(
        subset=['item_description'], keep='first')
    df.sort_values(by='pred', ascending=False,  inplace=True)
    df.to_csv('backprocess_test.csv')
    return df


def partial_predict_hk(df):    
    df.set_index('order_number', inplace=True)
    schduled_due_exist = df[df.order_scheduled_due.notna()]
    df.printed_due = schduled_due_exist.order_scheduled_due
    df.printed_due = pd.to_datetime(df.printed_due)
    df['shipping_process_ahead'] = False
    res = df.copy()

    with open('./etc/stage_mapper3.json') as json_file:
        stage_mapper = json.load(json_file)
        facility_index = np.array(stage_mapper["index"])
        facility_map = stage_mapper["map"]
        for row in res.itertuples():
            index = row[0]
            facility_id = row.facility_id
            facilities = np.array(row.facility.split(' '))
            cur_facility_index = np.where(facilities == facility_id)[0][0]
            mask = np.isin(facility_index, facilities)
            f_key = str(np.where(mask == True)[0])
            if ('H0030' in facilities) and (facility_id < 'H0070'):
                df.loc[index, 'shipping_process_ahead'] = True
            if cur_facility_index == 1:
                df.loc[index, 'ratio'] = 1
            else:
                ratio_arr = None
                if f_key in facility_map.keys():
                    ratio_arr = facility_map[f_key]
                else:
                    sub_key = difflib.get_close_matches(
                        f_key, facility_map.keys())[0]
                    ratio_arr = facility_map[sub_key]
                df.loc[index, 'ratio'] = sum(
                    ratio_arr[cur_facility_index - 1:])
            for f in facilities:
                res.loc[index, f] = 1
    for name in flag_list:
        res[name] = np.where(res['item_description'].str.count(name), 1, 0)
    res['product_family'] = res['product_family'].apply(
        lambda x: family_mapper(x))
    res['product_model'] = res['product_model'].apply(
        lambda x: model_mapper(x))
    res['reference_number'] = res['reference_number'].apply(
        lambda x: reference_mapper(x))
    res['product_length'] = res['product_length'].apply(lambda x: x / 1000)
    res['steps'] = res['facility'].apply(lambda x: len(x.split(' ')))
    res = res.reindex(set(res.columns.tolist() + cols_to_keep),
                      axis=1)  # version > 0.20.0
    res = res[cols_to_keep]
    res = res.fillna(0)
    val = res.astype(float)
    x = val.iloc[:, :].values
    v_xb = torch.tensor(x, dtype=torch.float32).to(dev)
    pred = model(v_xb)[:, 0].tolist()
    df['original_pred'] = pred / 24
    df['pred'] = pred * df['ratio']
    df['pred'] = df['pred'].astype('int')
    df['pred'] = df['pred'].apply(
        lambda x: timedelta(hours=x))
    df.loc[df['shipping_process_ahead'], 'pred'] = df['pred'] + timedelta(days=21)
    df['start_due'] = df['printed_due'] - df['pred']
    df.sort_values('start_due', inplace=True)
    df.to_csv('hk.csv')
    return df

all_priority_call_test()

In [84]:
kanban_df = get_kanban()
kanban_df
GY4_RAILS = ['SRS12', 'SRS12W', 'SRS15', 'SRS15W', 'SRS9X']

In [164]:
# stock_all()
srs_stock = stock.GY4.query('family.str.contains("SRS") & length >= 1000').copy()
srs_stock

,family,model,item_number,item_description,length,qty,gy,total_length
318,SRS,12,814012100C10000,SRS12-1000LM GROUND RAIL,1000.0,962.0,GY4,962.00
323,SRS,12,41401210001000A,SRS12-1000LM RAIL HALF RAIL 4,1000.0,33.0,GY4,33.00
324,SRS,12,41381220001000A,SRS12W-1000LM HALF RAIL 4,1000.0,13.0,GY4,13.00
325,SRS,12,41401210002000A,SRS12-2000LM RAIL HALF RAIL 4,2000.0,5.0,GY4,10.00
327,SRS,12W,41381215002000A,SRS12W-2000LM HALF RAIL 4,2000.0,94.0,GY4,188.00
329,SRS,15,814015150C10000,SRS15-1000LM GROUND RAIL,1000.0,284.0,GY4,284.00
331,SRS,15,41401515002000A,SRS15-2000LM HALF RAIL 4,2000.0,23.0,GY4,46.00
333,SRS,15,41401520001000A,SRS15-1000LM RAIL HALF RAIL 4,1000.0,13.0,GY4,13.00
334,SRS,15W,41381515002000A,SRS15W-2000LM HALF RAIL 4,2000.0,52.0,GY4,104.00
336,SRS,9X,814010075C10001,SRS9X-1000LM (GK)GROUND RAIL,1000.0,15.0,GY4,15.00


In [165]:
# srs_stock = srs_stock.query('model.str.contains("W") or family.str.contains("W")')
for r in srs_stock.itertuples():
    index = r[0]
    if r.model.endswith("W"):
        srs_stock.loc[index, 'model'] = r.model[:-1]
        srs_stock.loc[index, 'family'] = 'SRS-W'
srs_stock

,family,model,item_number,item_description,length,qty,gy,total_length
318,SRS,12,814012100C10000,SRS12-1000LM GROUND RAIL,1000.0,962.0,GY4,962.00
323,SRS,12,41401210001000A,SRS12-1000LM RAIL HALF RAIL 4,1000.0,33.0,GY4,33.00
324,SRS,12,41381220001000A,SRS12W-1000LM HALF RAIL 4,1000.0,13.0,GY4,13.00
325,SRS,12,41401210002000A,SRS12-2000LM RAIL HALF RAIL 4,2000.0,5.0,GY4,10.00
327,SRS-W,12,41381215002000A,SRS12W-2000LM HALF RAIL 4,2000.0,94.0,GY4,188.00
329,SRS,15,814015150C10000,SRS15-1000LM GROUND RAIL,1000.0,284.0,GY4,284.00
331,SRS,15,41401515002000A,SRS15-2000LM HALF RAIL 4,2000.0,23.0,GY4,46.00
333,SRS,15,41401520001000A,SRS15-1000LM RAIL HALF RAIL 4,1000.0,13.0,GY4,13.00
334,SRS-W,15,41381515002000A,SRS15W-2000LM HALF RAIL 4,2000.0,52.0,GY4,104.00
336,SRS,9X,814010075C10001,SRS9X-1000LM (GK)GROUND RAIL,1000.0,15.0,GY4,15.00


In [168]:
stock_sum = srs_stock.groupby(['family', 'model', 'length'])['total_length'].sum()
stock_sum = pd.DataFrame(stock_sum)
stock_sum = stock_sum.reset_index()
stock_sum['family'] = stock_sum['family'].astype(str)
stock_sum['model'] = stock_sum['model'].astype(str)

for r in stock_sum.itertuples():
    


,family,model,length,total_length
0,SRS,12,1000.0,1008.00
1,SRS,12,2000.0,10.00
2,SRS,15,1000.0,297.00
3,SRS,15,2000.0,46.00
4,SRS,9X,1000.0,39.00
5,SRS,9X,1240.0,1.24
6,SRS-W,12,1000.0,147.00
7,SRS-W,12,1430.0,25.74
8,SRS-W,12,2000.0,188.00
9,SRS-W,15,1000.0,321.00


In [162]:
demand_all()


srs_demand

,product_length,product_family,product_model,item_description,printed_due,qty,mos,over1000,total_meter
920,1189.0,SRS-W,12,SRS12W-1189LM RAIL,2023-04-10,2.0,M185272,True,2.378
924,1177.0,SRS-W,12,SRS12W-1177LM RAIL,2023-04-10,2.0,M185274,True,2.354
960,1110.0,SRS-W,12,SRS12W-1110LM RAIL,2023-04-21,10.0,"M197529, M197530",True,11.100
1074,945.0,SRS-W,15,SRS15W-945LM RAIL,2023-03-31,60.0,"M121981, M121975, M121976, M121982, M121974, M...",False,56.700
1141,865.0,SRS,12,SRS12-865LM RAIL,2023-04-11,16.0,"M181615, M181616",False,13.840
...,...,...,...,...,...,...,...,...,...
2343,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-06,1.0,M197706,False,0.110
2344,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-06,1.0,M197708,False,0.110
2345,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-07,1.0,M197710,False,0.110
2346,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-07,1.0,M197712,False,0.110


In [232]:
from util_func import get_mondays

def generate_gy4_demand(demand_gy4):
    mondays = get_mondays()
    mondays_str = [x.strftime("%Y-%m-%d") for x in mondays]
    srs_demand = demand_gy4.query('product_family.str.startswith("SRS")').copy()
    srs_demand = srs_demand[srs_demand['printed_due'] < date.today() + timedelta(weeks=15)]
    srs_demand['over1000'] = srs_demand['product_length'] > 1000
    srs_demand['total_meter'] = srs_demand['product_length'] * srs_demand['qty'] / 1000
    srs_cols_to_keep = ['product_length', 'product_family', 'product_model', 'item_description', 'printed_due', 'qty', 'mos', 'over1000', 'total_meter']
    srs_demand = srs_demand[srs_cols_to_keep]
    srs_demand['demand_week'] = srs_demand['printed_due'].apply(lambda x: next(i for i in mondays_str[::-1] if i <= x.strftime("%Y-%m-%d")))
    srs_demand['total_meter'] = srs_demand['total_meter'].astype('int')
    srs_demand_sum = srs_demand.groupby(['product_family', 'product_model', 'over1000', 'demand_week'])['total_meter'].sum()
    srs_demand_sum = srs_demand_sum.reset_index()
    srs_demand_sum
    srs_demand_sum['type'] = srs_demand_sum['product_family'] + srs_demand_sum['product_model'] + srs_demand_sum['over1000'].astype(str)
    pivoted = srs_demand_sum.pivot(index='type', columns='demand_week', values='total_meter')

    pivoted_cols = pivoted.columns
    for week in mondays_str:
        if week not in pivoted_cols:
            pivoted[week] = None

    pivoted = pivoted[mondays_str]
    pivoted['total'] = pivoted.sum(axis=1)
    pivoted = pivoted.fillna(0)
    return pivoted
generate_gy4_demand(demand.GY4)

demand_week,2023-02-20,2023-02-27,2023-03-06,2023-03-13,2023-03-20,2023-03-27,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-01,2023-05-08,2023-05-15,2023-05-22,2023-05-29,2023-06-05,total
type,,,,,,,,,,,,,,,,,
SRS-W12False,0,0,0,0,0,0.0,0.0,14.0,36.0,22.0,0,0,0,0,0,0,72.0
SRS-W12True,0,0,0,0,0,0.0,0.0,4.0,11.0,0.0,0,0,0,0,0,0,15.0
SRS-W15False,0,0,0,0,0,56.0,1.0,2.0,5.0,9.0,0,0,0,0,0,0,73.0
SRS-W9False,0,0,0,0,0,0.0,0.0,1.0,15.0,0.0,0,0,0,0,0,0,16.0
SRS12False,0,0,0,0,0,0.0,73.0,80.0,100.0,209.0,0,0,0,0,0,0,462.0
SRS15False,0,0,0,0,0,0.0,0.0,0.0,3.0,1.0,0,0,0,0,0,0,4.0


In [219]:
srs_demand.columns

Index(['product_length', 'product_family', 'product_model', 'item_description',
       'printed_due', 'qty', 'mos', 'over1000', 'total_meter', 'demand_week'],
      dtype='object')

In [167]:
from util_func import get_mondays

mondays = get_mondays()
mondays_date = [date.fromordinal(x.toordinal()) for x in mondays]
mondays_date.reverse()
# mondays_date
print(mondays_date)
srs_demand['demand_week'] = srs_demand['printed_due'].apply(lambda x: next(i for i in mondays_date if i <= x))
# srs_demand['total_meter'] = srs_demand['total_meter'].astype('int')
srs_demand_sum = srs_demand.groupby(['product_family', 'product_model', 'over1000'])['total_meter'].sum()
srs_demand_sum = srs_demand_sum.reset_index()
# print(srs_demand_sum)
mondays = get_mondays()
# print(mondays)
for r in srs_demand_sum.itertuples():
    family = r.product_family
    model = r.product_model
    temp_demand = r.total_meter    
    target_df = stock_length = stock_sum[(stock_sum['family'] == family) & (stock_sum['model'] == model)]
    if  r.over1000 == False:
        temp_stock = target_df[target_df['length'] == 1000]['total_length'].sum()
    else:        
        temp_stock = target_df[target_df['length'] > 1000]['total_length'].sum()
    # if temp_demand > temp_stock:
    print(family, model, temp_demand, temp_stock, r.over1000)


SRS 12 466.49699999999996 1008.0 False
SRS 15 6.0 297.0 False
SRS-W 12 84.125 147.0 False
SRS-W 12 15.832 213.74 True
SRS-W 15 79.364 321.0 False
SRS-W 9 16.2 678.0 False


In [160]:
pd.unique(srs_demand['product_model'])

array(['12', '15', '9'], dtype=object)

In [202]:
# mondays_str = [x.strftime("%Y-%m-%d") for x in mondays]

# srs_demand['due_week'] = srs_demand['printed_due'].apply(lambda x: x)

[datetime.date(2023, 6, 5), datetime.date(2023, 5, 29), datetime.date(2023, 5, 22), datetime.date(2023, 5, 15), datetime.date(2023, 5, 8), datetime.date(2023, 5, 1), datetime.date(2023, 4, 24), datetime.date(2023, 4, 17), datetime.date(2023, 4, 10), datetime.date(2023, 4, 3), datetime.date(2023, 3, 27), datetime.date(2023, 3, 20), datetime.date(2023, 3, 13), datetime.date(2023, 3, 6), datetime.date(2023, 2, 27), datetime.date(2023, 2, 20)]


,product_length,product_family,product_model,item_description,printed_due,qty,mos,over1000,total_meter,demand_week
920,1189.0,SRS-W,12,SRS12W-1189LM RAIL,2023-04-10,2.0,M185272,True,2.378,2023-04-10
924,1177.0,SRS-W,12,SRS12W-1177LM RAIL,2023-04-10,2.0,M185274,True,2.354,2023-04-10
960,1110.0,SRS-W,12,SRS12W-1110LM RAIL,2023-04-21,10.0,"M197529, M197530",True,11.100,2023-04-17
1074,945.0,SRS-W,15,SRS15W-945LM RAIL,2023-03-31,60.0,"M121981, M121975, M121976, M121982, M121974, M...",False,56.700,2023-03-27
1141,865.0,SRS,12,SRS12-865LM RAIL,2023-04-11,16.0,"M181615, M181616",False,13.840,2023-04-10
...,...,...,...,...,...,...,...,...,...,...
2343,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-06,1.0,M197706,False,0.110,2023-04-03
2344,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-06,1.0,M197708,False,0.110,2023-04-03
2345,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-07,1.0,M197710,False,0.110,2023-04-03
2346,110.0,SRS-W,15,SRS15W-110LM RAIL,2023-04-07,1.0,M197712,False,0.110,2023-04-03


In [188]:
mondays_str.reverse()
next(i for i in mondays_str if i <= '2023-03-21')

'2023-03-20'

In [182]:
reversed_mondays

In [184]:
mondays_str[::-1]


['2023-02-20',
 '2023-02-27',
 '2023-03-06',
 '2023-03-13',
 '2023-03-20',
 '2023-03-27',
 '2023-04-03',
 '2023-04-10',
 '2023-04-17',
 '2023-04-24',
 '2023-05-01',
 '2023-05-08',
 '2023-05-15',
 '2023-05-22',
 '2023-05-29',
 '2023-06-05']